In [1]:
import math
import random
import torch
import torchvision
import time 
from torchvision import datasets ,transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader,Dataset
from torch import nn 
from torch.nn.functional import relu ,softmax 
import copy 
from torch.utils.data import Subset
import pandas as pd 
import numpy as np
from PIL import Image
import pyswarms as ps 


In [2]:

data_transform=transforms.Compose ([transforms.ToTensor()])
class light_source_dataset(Dataset):
    def __init__(self,path,transform):
        self.df_data=pd.read_csv(path)
        self.df_data['Classifier'] = pd.Categorical(pd.factorize(self.df_data['Classifier'])[0])
        self.labels=np.asarray(self.df_data.iloc[:,self.df_data.shape[1]-1])
        self.image_as_np=np.asarray(self.df_data.iloc[:,0:self.df_data.shape[1]-1]).astype('uint8')
        self.trans=transform
        
    def __len__(self):
        return len(self.df_data.index)
    def __getitem__(self,index):
        image_np=self.image_as_np[index,:,None]
       
        pillow_image=Image.fromarray(image_np.astype('uint8'))
        
        single_label=self.labels[index]
        if (self.trans is not None):
            img_as_tensor=self.trans(pillow_image)
        
        
            
        return (img_as_tensor,single_label)
        
dataset=light_source_dataset('WaveformEW/WaveformEW.csv',data_transform)
print(len(dataset))
train_size=int (len(dataset)*0.6)
test_size=len(dataset)-train_size
trainloader=DataLoader(dataset,batch_size=1)
torch.manual_seed(1)
train_data,test_data=torch.utils.data.random_split(dataset,[int (train_size), int (test_size)])
dataloader={'train':DataLoader(train_data,shuffle=False ,batch_size=16),
            'val':DataLoader(test_data,shuffle=False,batch_size=16
                            )}

dataset_sizes={'train':len(train_data),
               'val':len(test_data)}

device=torch.device("cuda:0"if torch.cuda.is_available () else "cpu")
print(dataset_sizes["train"])
print (dataset_sizes["val"])

 
    



5000
3000
2000


In [3]:

class Model(torch.nn.Module):
    def __init__(self):
        super (Model,self ).__init__()
        self .linear1=nn.Linear (40,100)
        self.linear2=nn.Linear (100,3)
        
    def Forward (self,x,mask,p):
        #feed forward function 
        x=x.view(-1,40)
        act1=relu(self.linear1(x))
        act1_masked =self.masking (act1,mask,p)
        act2=softmax(self.linear2(act1_masked ))
        return act2
    
    
    
    def masking (self,act1,mask,p):
        if (self.training ==True ):
            return ((act1*mask)/p)
        else :
            return (act1)
        



  
  
  


      

In [4]:
def train_model(model, criterion, optimizer,mask,p, num_epochs=25):
  # the mask has to be specified 

      mask=torch.from_numpy(mask)
      since = time.time()

      best_model_wts = copy.deepcopy(model.state_dict())
      best_acc = 0.0
      losses=[]
      accuracies=[]

      for epoch in range(num_epochs):
          print('Epoch {}/{}'.format(epoch, num_epochs - 1))
          print('-' * 10)

          # Each epoch has a training and validation phase
          for phase in ['train', 'val']:
              if phase == 'train':
                  
                  model.train()  # Set model to training mode
              else:

                  model.eval()   # Set model to evaluate mode

              running_loss = 0.0
              running_corrects = 0

              # Iterate over data.
              for inputs, labels in dataloader[phase]:
                inputs=inputs.to(device)
                labels=labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs= model.Forward(inputs,mask,p)

                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)# backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        #print (model.linear1.weight.grad)
                        #print (model.linear2.weight.grad)

                  # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                  

              epoch_loss = running_loss / dataset_sizes[phase]
              epoch_acc = running_corrects.double() / dataset_sizes[phase]
            

              print('{} Loss: {:.4f} Acc: {:.4f} '.format(
                  phase, epoch_loss, epoch_acc))

              # deep copy the model
              if phase == 'val' and epoch_acc > best_acc:
                  best_acc = epoch_acc
                  best_model_wts = copy.deepcopy(model.state_dict())
              if phase=='train':
                losses.append(epoch_loss)
                accuracies.append(epoch_acc)


          

      time_elapsed = time.time() - since
      print('Training complete in {:.0f}m {:.0f}s'.format(
          time_elapsed // 60, time_elapsed % 60))
      print('Best val Acc: {:4f}'.format(best_acc))

      # load best model weights
      model.load_state_dict(best_model_wts)
      return model,losses,accuracies

In [5]:
def fitness(mask):
        # finding the fitness of a particular mask
        #accuracy of all training set is the fitness in one epoch
        #putting model in train mode
        mask=sigmoid_new(mask)
        mask=torch.from_numpy(mask)
        running_loss=0
        model.train()
        
        for inputs,labels in dataloader ['train']:
            inputs=inputs.to(device)
            labels=labels.to(device )
            outputs=model.Forward(inputs,mask,keep_prob(mask))
            _,preds=torch.max(outputs,1)
            loss=criterion (outputs,labels)
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / dataset_sizes['train']
        return epoch_loss

    
    
    
def f(x):
    #Higher-level method to do forward_prop in the whole swarm.
    #Inputs:x: numpy.ndarray of shape (n_particles, dimensions) The swarm that will perform the search
    #Returns numpy.ndarray of shape (n_particles, )  The computed loss for each particle
    
    
    
    j = [fitness(x[i,:]) for i in range(num_particles)]
    return np.array(j)




def keep_prob (mask):

        num_one =0
        for i in range (maskLength):
            if (mask[i]==1):
                num_one=num_one+1
        return float(num_one/maskLength)
def sigmoid_new(x):
    z = 1/(1 + np.exp(-x))
    z=(z>0.5).astype('float32')
    return z



In [6]:
options = {'c1': 2, 'c2': 2, 'w':0.4}
num_particles=30
maskLength=100
torch.manual_seed(6)
model=Model()
model.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.9)



In [7]:
epochgens=0
total_loss=[]
total_acc=[]
mask_par=torch.bernoulli(torch.empty(num_particles,maskLength).uniform_(0,1)).numpy()
optim = ps.single.GlobalBestPSO(n_particles=num_particles, dimensions=maskLength, options=options,init_pos=mask_par)
while (epochgens<=20):
    print ('Epoch generations (',epochgens,'/200)',end=' :')
    cost,mask = optim.optimize(f,  iters=50)
    mask=sigmoid_new(mask)
    print('keep prob',keep_prob(mask))
    model,losses,accuracies=train_model(model,criterion,optimizer,mask,keep_prob(mask),30)
    total_loss=total_loss+losses
    total_acc=total_acc+accuracies
    epochgens+=1

2020-04-10 16:27:00,508 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 2, 'c2': 2, 'w': 0.4}
pyswarms.single.global_best:   0%|                                                                                |0/50

Epoch generations ( 0 /200) :

D:\ana\envs\packt_torch\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':
pyswarms.single.global_best:  90%|████████████████████████████████████████████████████████▋      |45/50, best_cost=1.09D:\ana\envs\packt_torch\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: overflow encountered in exp
pyswarms.single.global_best: 100%|███████████████████████████████████████████████████████████████|50/50, best_cost=1.09
2020-04-10 16:44:27,075 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1.0942221593856811, best pos: [-8.8512602e+00 -9.9848309e+00  9.1996640e-01 -3.3152934e-02
 -5.7579327e+00  3.2401814e+00  3.4489696e+00  2.4132452e+00
  1.5718050e+00 -7.7564836e+00  7.9179635e+00 -1.0397106e+01
  1.4418080e+00  1.3985373e+00  7.3433501e-01  1.1687936e+00
 -2.2276324e-01 -3.8587027e+00  1.0844530e-01  1.6425638e+01
 -1

keep prob 0.39
Epoch 0/29
----------
train Loss: 1.0936 Acc: 0.3837 
val Loss: 1.0986 Acc: 0.3285 
Epoch 1/29
----------
train Loss: 1.0919 Acc: 0.3977 
val Loss: 1.0978 Acc: 0.3475 
Epoch 2/29
----------
train Loss: 1.0899 Acc: 0.4113 
val Loss: 1.0970 Acc: 0.3585 
Epoch 3/29
----------
train Loss: 1.0875 Acc: 0.4220 
val Loss: 1.0960 Acc: 0.3820 
Epoch 4/29
----------
train Loss: 1.0847 Acc: 0.4320 
val Loss: 1.0949 Acc: 0.3975 
Epoch 5/29
----------
train Loss: 1.0814 Acc: 0.4490 
val Loss: 1.0935 Acc: 0.4165 
Epoch 6/29
----------
train Loss: 1.0775 Acc: 0.4537 
val Loss: 1.0919 Acc: 0.4315 
Epoch 7/29
----------
train Loss: 1.0729 Acc: 0.4617 
val Loss: 1.0899 Acc: 0.4440 
Epoch 8/29
----------
train Loss: 1.0675 Acc: 0.4727 
val Loss: 1.0876 Acc: 0.4590 
Epoch 9/29
----------
train Loss: 1.0614 Acc: 0.4833 
val Loss: 1.0849 Acc: 0.4750 
Epoch 10/29
----------
train Loss: 1.0546 Acc: 0.4893 
val Loss: 1.0819 Acc: 0.4815 
Epoch 11/29
----------
train Loss: 1.0475 Acc: 0.4953 
val L

2020-04-10 16:44:53,581 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 2, 'c2': 2, 'w': 0.4}
pyswarms.single.global_best:   0%|                                                                                |0/50

...

keep prob 0.19
Epoch 0/29
----------
train Loss: 0.9424 Acc: 0.6040 
val Loss: 1.0188 Acc: 0.5550 
Epoch 1/29
----------
train Loss: 0.9422 Acc: 0.6040 
val Loss: 1.0187 Acc: 0.5555 
Epoch 2/29
----------
train Loss: 0.9420 Acc: 0.6043 
val Loss: 1.0186 Acc: 0.5560 
Epoch 3/29
----------
train Loss: 0.9418 Acc: 0.6040 
val Loss: 1.0185 Acc: 0.5560 
Epoch 4/29
----------
train Loss: 0.9416 Acc: 0.6040 
val Loss: 1.0183 Acc: 0.5555 
Epoch 5/29
----------
train Loss: 0.9414 Acc: 0.6043 
val Loss: 1.0182 Acc: 0.5555 
Epoch 6/29
----------
train Loss: 0.9412 Acc: 0.6040 
val Loss: 1.0181 Acc: 0.5540 
Epoch 7/29
----------
train Loss: 0.9410 Acc: 0.6040 
val Loss: 1.0180 Acc: 0.5540 
Epoch 8/29
----------
train Loss: 0.9409 Acc: 0.6040 
val Loss: 1.0178 Acc: 0.5535 
Epoch 9/29
----------
train Loss: 0.9407 Acc: 0.6043 
val Loss: 1.0178 Acc: 0.5535 
Epoch 10/29
----------
train Loss: 0.9405 Acc: 0.6033 
val Loss: 1.0177 Acc: 0.5540 
Epoch 11/29
----------
train Loss: 0.9403 Acc: 0.6040 
val L

2020-04-10 19:39:20,792 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 2, 'c2': 2, 'w': 0.4}
pyswarms.single.global_best:   0%|                                                                                |0/50

val Loss: 1.0163 Acc: 0.5560 
Training complete in 0m 22s
Best val Acc: 0.556000
Epoch generations ( 14 /200) :

pyswarms.single.global_best:  28%|█████████████████▋                                             |14/50, best_cost=0.94

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(range(630),total_loss,label='loss')
plt.plot(range(630),total_acc,label='accuracy')
plt.labelegend()
plt.xlabel('number of epochs ')
plt.title('drop_pso')
plt.savefig('wave 100/pso_drop.jpg')

In [14]:
mask=torch.ones((100,)).numpy().astype('float32')
mask

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)

In [15]:
model,losses,accuracies=train_model(model,criterion,optimizer,mask,1.0,300)

Epoch 0/299
----------


D:\ana\envs\packt_torch\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


train Loss: 1.0042 Acc: 0.5950 
val Loss: 1.0140 Acc: 0.5660 
Epoch 1/299
----------
train Loss: 1.0001 Acc: 0.5973 
val Loss: 1.0110 Acc: 0.5690 
Epoch 2/299
----------
train Loss: 0.9972 Acc: 0.5970 
val Loss: 1.0085 Acc: 0.5695 
Epoch 3/299
----------
train Loss: 0.9947 Acc: 0.5967 
val Loss: 1.0063 Acc: 0.5725 
Epoch 4/299
----------
train Loss: 0.9925 Acc: 0.5947 
val Loss: 1.0043 Acc: 0.5740 
Epoch 5/299
----------
train Loss: 0.9904 Acc: 0.5933 
val Loss: 1.0024 Acc: 0.5730 
Epoch 6/299
----------
train Loss: 0.9885 Acc: 0.5940 
val Loss: 1.0007 Acc: 0.5735 
Epoch 7/299
----------
train Loss: 0.9867 Acc: 0.5937 
val Loss: 0.9990 Acc: 0.5740 
Epoch 8/299
----------
train Loss: 0.9850 Acc: 0.5930 
val Loss: 0.9975 Acc: 0.5730 
Epoch 9/299
----------
train Loss: 0.9834 Acc: 0.5933 
val Loss: 0.9960 Acc: 0.5730 
Epoch 10/299
----------
train Loss: 0.9819 Acc: 0.5923 
val Loss: 0.9946 Acc: 0.5740 
Epoch 11/299
----------
train Loss: 0.9805 Acc: 0.5917 
val Loss: 0.9933 Acc: 0.5755 
E

val Loss: 0.9688 Acc: 0.5775 
Epoch 96/299
----------
train Loss: 0.9504 Acc: 0.5943 
val Loss: 0.9688 Acc: 0.5770 
Epoch 97/299
----------
train Loss: 0.9502 Acc: 0.5943 
val Loss: 0.9688 Acc: 0.5775 
Epoch 98/299
----------
train Loss: 0.9501 Acc: 0.5943 
val Loss: 0.9688 Acc: 0.5770 
Epoch 99/299
----------
train Loss: 0.9500 Acc: 0.5947 
val Loss: 0.9687 Acc: 0.5770 
Epoch 100/299
----------
train Loss: 0.9499 Acc: 0.5947 
val Loss: 0.9687 Acc: 0.5770 
Epoch 101/299
----------
train Loss: 0.9498 Acc: 0.5947 
val Loss: 0.9687 Acc: 0.5765 
Epoch 102/299
----------
train Loss: 0.9497 Acc: 0.5950 
val Loss: 0.9687 Acc: 0.5750 
Epoch 103/299
----------
train Loss: 0.9496 Acc: 0.5950 
val Loss: 0.9687 Acc: 0.5750 
Epoch 104/299
----------
train Loss: 0.9495 Acc: 0.5950 
val Loss: 0.9686 Acc: 0.5750 
Epoch 105/299
----------
train Loss: 0.9494 Acc: 0.5950 
val Loss: 0.9686 Acc: 0.5750 
Epoch 106/299
----------
train Loss: 0.9493 Acc: 0.5947 
val Loss: 0.9686 Acc: 0.5745 
Epoch 107/299
---

train Loss: 0.9429 Acc: 0.5997 
val Loss: 0.9684 Acc: 0.5720 
Epoch 191/299
----------
train Loss: 0.9428 Acc: 0.6000 
val Loss: 0.9684 Acc: 0.5720 
Epoch 192/299
----------
train Loss: 0.9427 Acc: 0.6000 
val Loss: 0.9684 Acc: 0.5720 
Epoch 193/299
----------
train Loss: 0.9427 Acc: 0.5997 
val Loss: 0.9684 Acc: 0.5720 
Epoch 194/299
----------
train Loss: 0.9426 Acc: 0.5997 
val Loss: 0.9684 Acc: 0.5725 
Epoch 195/299
----------
train Loss: 0.9425 Acc: 0.5997 
val Loss: 0.9684 Acc: 0.5725 
Epoch 196/299
----------
train Loss: 0.9425 Acc: 0.6000 
val Loss: 0.9685 Acc: 0.5720 
Epoch 197/299
----------
train Loss: 0.9424 Acc: 0.6000 
val Loss: 0.9685 Acc: 0.5725 
Epoch 198/299
----------
train Loss: 0.9423 Acc: 0.6000 
val Loss: 0.9685 Acc: 0.5730 
Epoch 199/299
----------
train Loss: 0.9423 Acc: 0.6000 
val Loss: 0.9685 Acc: 0.5730 
Epoch 200/299
----------
train Loss: 0.9422 Acc: 0.6007 
val Loss: 0.9685 Acc: 0.5730 
Epoch 201/299
----------
train Loss: 0.9421 Acc: 0.6007 
val Loss: 0

val Loss: 0.9697 Acc: 0.5675 
Epoch 285/299
----------
train Loss: 0.9369 Acc: 0.6037 
val Loss: 0.9697 Acc: 0.5670 
Epoch 286/299
----------
train Loss: 0.9368 Acc: 0.6037 
val Loss: 0.9697 Acc: 0.5670 
Epoch 287/299
----------
train Loss: 0.9367 Acc: 0.6037 
val Loss: 0.9697 Acc: 0.5670 
Epoch 288/299
----------
train Loss: 0.9367 Acc: 0.6037 
val Loss: 0.9698 Acc: 0.5670 
Epoch 289/299
----------
train Loss: 0.9366 Acc: 0.6037 
val Loss: 0.9698 Acc: 0.5670 
Epoch 290/299
----------
train Loss: 0.9366 Acc: 0.6037 
val Loss: 0.9698 Acc: 0.5670 
Epoch 291/299
----------
train Loss: 0.9365 Acc: 0.6033 
val Loss: 0.9698 Acc: 0.5670 
Epoch 292/299
----------
train Loss: 0.9364 Acc: 0.6033 
val Loss: 0.9698 Acc: 0.5665 
Epoch 293/299
----------
train Loss: 0.9364 Acc: 0.6037 
val Loss: 0.9698 Acc: 0.5655 
Epoch 294/299
----------
train Loss: 0.9363 Acc: 0.6043 
val Loss: 0.9698 Acc: 0.5655 
Epoch 295/299
----------
train Loss: 0.9362 Acc: 0.6043 
val Loss: 0.9699 Acc: 0.5655 
Epoch 296/299